# 对于2018年CIC2rd的反馈数据进行文本分类

## install dependency and initial session and S3 bucket

In [50]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3
from random import shuffle

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = 'ray-ai-ml-bjs' #sess.default_bucket() # Replace with your own bucket name if needed
print(bucket)
prefix = 'classification/blazingtext/bmwticket' 

arn:aws-cn:iam::876820548815:role/Sagemaker-Bootcamp-SageMakerExecutionRole-Z3VF78G260T1
ray-ai-ml-bjs


In [51]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple jieba --upgrade
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple paddlepaddle-tiny

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Requirement already up-to-date: jieba in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (0.42.1)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## Prepare classification category
- All return '\n' convert to `_` for example `LSC\n远程指令` convert to `LSC_远程指令`
- Convert alphabet to Uppercase

In [52]:
import jieba
import re
import pandas as pd
import os
import csv

## 读入原始Excel文件，并通过参数 usecols 提取相关列，保存为csv文件，作为算法Raw data输入

In [53]:
excel_data_df = pd.read_excel('20201026_customer feedback_with comments.xlsx', sheet_name='Sheet1', usecols=['channel', 'Rating', 'Customer Comment', 'Positive or Negative', 'Coding level 1', 'Coding level 2'], index_col=None)
#print(excel_data_df[0:5])

raw_app_comments_file = 'raw-app-comments-1030.csv'
excel_data_df.to_csv(raw_app_comments_file, index=False)
sentiment_input_with_def = []
sentiment_input_without_def = []
sentiment_def= []
input_with_category = []
input_without_category = []
data_offset_counts = {}
category_def = []

csv_data_df = pd.read_csv(raw_app_comments_file, delimiter=',', index_col=None)
#print(csv_data_df[0:10])

## 处理分类, 如果分类有更新，需要重新运行文件 取消注释 modify_label_file，重新运行
 - 分类标签处理会去掉换行回车键
 - 多分类标签，标签之间通过'__'拼接为一条
 - 字母转换为大写字母
 - 标签格式：数字,标签字符串，例如1154,互联驾驶使用年限__USABILITY/DIFFICULT TO USE

In [61]:
def handle_category_str(coding_level1, coding_level2):
    if (coding_level1!='' and pd.notnull(coding_level1)) and (coding_level2!='' and pd.notnull(coding_level2)):
        category_str = str(coding_level1).split('\n', 1)[0] + "__" + str(coding_level2).split('\n', 1)[0]
        category_str = category_str.upper()
    elif (coding_level1=='' or pd.isnull(coding_level1)):
        category_str = str(coding_level2).split('\n', 1)[0]
    elif (coding_level2=='' or pd.isnull(coding_level2)):
        category_str = str(coding_level1).split('\n', 1)[0]
    else:
        category_str = 'No_Category'
    return category_str


def modify_label_file(csv_data_df):
    for index, data in csv_data_df.iterrows():
        coding_level1 = data['Coding level 1']
        coding_level2 = data['Coding level 2']
        category_str = handle_category_str(coding_level1, coding_level2)
        category_def.append(category_str)

    category_def_unique = list(set(category_def))
    print(category_def_unique)
    temp_category_id = 1047
    for x in category_def_unique:
        category_id = temp_category_id
        print(category_id,",",x)
        temp_category_id +=1

#modify_label_file(csv_data_df)
        
index_to_label = {} #1001:APP显示问题
label_to_index = {} #APP显示问题:1001
with open("app-comments-multiple-label.txt") as f:
    for i,label in enumerate(f.readlines()):
        ll = label.strip().split(',')
        index_to_label[ll[0]] = ll[1].upper()
        label_to_index[ll[1].upper()] = ll[0]
print(label_to_index)

{'APP显示问题': '1001', 'APP版本问题': '1002', 'APP登陆问题': '1003', 'APP车辆信息显示问题': '1004', 'BON': '1005', 'CARLIFE': '1006', 'CARPLAY': '1007', 'ETC': '1008', 'GPS': '1009', 'IBA用户手册': '1010', 'LSC': '1011', 'LSC_远程指令': '1012', 'PIN码': '1013', 'POI': '1014', 'QQ音乐': '1015', 'RSU': '1016', 'RTTI': '1017', 'SIM卡相关': '1019', 'WIFI': '1020', '个性化设置': '1021', '二手车预激活问题': '1022', '会员': '1072', '其他': '1024', '即时充电': '1025', '喜马拉雅': '1026', '地图自动更新服务': '1027', '天猫精灵': '1028', '实时路况功能': '1029', '意见建议': '1030', '数字钥匙': '1031', '智慧停车': '1032', '更新BMW服务': '1033', '满意': '1034', '节日祝福': '1035', '行程摘要': '1036', '行车摘要': '1037', '车机端登录': '1038', '远程3D视图': '1039', '远程指令': '1040', '远程指令_LSC': '1041', '远程服务': '1042', '远程软件升级': '1043', '违章代缴': '1044', '预开通激活': '1045', '预激活开通': '1046', '其他/APP__OTHER': '1047', 'APP/社群__USABILITY/DIFFICULT TO USE': '1048', '车机端BMW CONNECTED/BMW云端互联__LOGINFAULT': '1049', '其他/蓝牙__CONNECTION FAULT': '1050', '其他/APP__USABILITY/DIFFICULT TO USE': '1051', '车机端BMW CONNECTED/BMW云端互联__USABILIT

## 处理情感数据
- Rule of NPS,
    - Positive: 10-9
    - Neutral: 8-7
    - Negative: 6-1
 
- Rule of App store:
    - Positive Reviews: 5-4
    - Negative Reviews: 3-1

In [62]:
def get_sentiment(channel, rating, sentiment):
    sentiment_switcher = { 
        "正向": "Positive", 
        "中性": "Neutral", 
        "负向": "Negative", 
    } 
    return_sentiment = sentiment_switcher.get(sentiment, None) 
    if channel == 'NPS':
        if rating >=9:
            return_sentiment = 'Positive'
        elif rating >=7:
            return_sentiment = 'Neutral'
        else:
            return_sentiment = 'Negative'
    elif channel.endswith("App Store"):
        if rating >=4:
            return_sentiment = 'Positive'
        else:
            return_sentiment = 'Negative'
    return return_sentiment

def unit_test_get_sentiment():
    # Unit Test for get_sentiment
    print("NPS")
    print(get_sentiment("NPS", 9, "Neutral"))
    print(get_sentiment("NPS", 9, "Negative"))
    print(get_sentiment("NPS", 7, "Positive"))
    print(get_sentiment("NPS", 8, "Negative"))
    print(get_sentiment("NPS", 6, "Positive"))
    print(get_sentiment("NPS", 5, "Negative"))
    print(get_sentiment("NPS", 4, "Neutral"))
    print("App Store")
    print(get_sentiment("IOS App Store", 4, "Neutral"))
    print(get_sentiment("Andior App Store", 5, "Negative"))
    print(get_sentiment("HuaWeiApp Store", 3, "Positive"))
    print(get_sentiment("Oppp App Store", 2, "Negative"))
    print(get_sentiment("Xiao Mi App Store", 1, "Positive"))
    print("feedback")
    print(get_sentiment("feedback", 0, "正向"))
    print(get_sentiment("feedback", 0, "中性"))
    print(get_sentiment("feedback", 0, "负向"))

# Unit Test
#unit_test_get_sentiment()

NPS
Positive
Positive
Neutral
Neutral
Negative
Negative
Negative
App Store
Positive
Positive
Negative
Negative
Negative
feedback
Positive
Neutral
Negative


## 将原始数据进行预处理
- 提取出分类，情感，评论，渠道，评分等
- 分为有分类和无分类
- 分为有情感类比和无情感类比
- 将分类标签格式转换为：index_!_分类标签_!_评论，例如 1011_!_LSC_!_定位不准，乱定位

In [70]:
category_def.clear()
input_without_category.clear()
input_with_category.clear()
sentiment_input_without_def.clear()
sentiment_input_with_def.clear()
sentiment_def.clear()
data_offset_counts.clear()

for index, data in csv_data_df.iterrows():
    channel_str = data['channel']
    rating_num = data.get('Rating',0)
    comment_str = data['Customer Comment']
    sentiment_str = data['Positive or Negative']
    coding_level1 = data['Coding level 1']
    coding_level2 = data['Coding level 2']
    category_str = handle_category_str(coding_level1, coding_level2)
    if (category_str == 'No_Category' or category_str =='nan'):
        input_without_category.append(str(comment_str))
        #print(channel_str, rating_num, comment_str, sentiment_str, coding_level1, coding_level2)
    else:
        category_def.append(category_str)
        category_index = label_to_index.get(category_str, None)
        input_with_category.append(str(category_index) + '_!_' + str(category_str) + '_!_' + str(comment_str))
        data_offset_counts[category_str] = data_offset_counts.get(category_str, 0) + 1
    
    sentiment_result = get_sentiment(channel_str, rating_num, sentiment_str)
    if sentiment_str==None or pd.isnull(sentiment_result):
        sentiment_input_without_def.append(str(comment_str))
    else:
        sentiment_input_with_def.append(str(comment_str))
        sentiment_def.append(str(sentiment_str))


## 预览原始数据预处理结果

In [71]:
print(input_with_category[0:3],'\n')
print(input_without_category[0:3],'\n')
print('Excel Sheet to CSV done - csv_data_df: ',len(csv_data_df), ' , input_with_category: ', len(input_with_category), ' , input_without_category: ', len(input_without_category), '\n')

print('data offset counts: ')
items = list(data_offset_counts.items())
items.sort(key=lambda x: x[1], reverse=True)
for i in range(30):
    word, count = items[i]
    print("{:<10}{:>7}".format(word, count))

print('sentiment: ',len(csv_data_df), ' , sentiment_input_with_def: ', len(sentiment_input_with_def), ' , sentiment_input_without_def: ', len(sentiment_input_without_def), '\n')
print('\n','sentiment_input with definition: ', len(sentiment_input_with_def), 'preview 10 items')
print(sentiment_input_with_def[0:10])
    

['1173_!_其他__OTHER_!_新的APP', '1173_!_其他__OTHER_!_good job', '1173_!_其他__OTHER_!_145666'] 

['v', '10', '2'] 

Excel Sheet to CSV done - csv_data_df:  7578  , input_with_category:  6941  , input_without_category:  637 

data offset counts: 
其他__OTHER    3329
其他/APP__USABILITY/DIFFICULT TO USE    897
APP车辆信息显示__USABILITY/DIFFICULT TO USE    294
远程指令__USABILITY/DIFFICULT TO USE    228
其他/APP__CONVENIENCE    206
其他__USABILITY/DIFFICULT TO USE    163
其他/APP__OTHER    163
预激活__USABILITY/DIFFICULT TO USE    154
其他/APP__DESIGN / VISUALS (WEB)    152
APP车辆信息显示__UPDATE NOT POSSIBLE    142
远程3D视图__USABILITY/DIFFICULT TO USE    117
会员__OTHER     100
预开通激活__USABILITY/DIFFICULT TO USE     88
意见建议-CONCERN&ADVICE__CONCERN&ADVICE     51
其他/APP__CONNECTION FAULT     49
其他__DESIGN / VISUALS (WEB)     48
定位__USABILITY/DIFFICULT TO USE     46
数字钥匙功能__USABILITY/DIFFICULT TO USE     46
其他/APP__UPDATE NOT POSSIBLE     44
互联驾驶信息更改__USABILITY/DIFFICULT TO USE     42
APP功能显示__USABILITY/DIFFICULT TO USE     38
其他

## 调用AWS comprehend API 进行情感分析
1. 示例检查API是否工作

In [72]:
session = boto3.Session(profile_name='global', region_name='us-east-1')
comprehend_client = session.client('comprehend')
sample_list = ['喜马拉雅FM可以像QQ音乐一样直接在车上应用么？还是只能在CARPLAY上使用', '你好，为什么在互联驾驶这个功能中定位不了我车了，显示说我在车上的GPS关闭了，但是我检查过了，也开着啊什么问题呢', '我喜欢使用互联驾驶这个功能中路线规划']
for s_input in sample_list:
    sentiment_response = comprehend_client.detect_sentiment(Text=s_input, LanguageCode='zh')
    sentiment_str = sentiment_response['Sentiment']
    sentiment_score = json.dumps(sentiment_response['SentimentScore'])
    print(s_input + ' - sentiment: ' + sentiment_str + ' - sentiment_score: ' + sentiment_score + '\n')


喜马拉雅FM可以像QQ音乐一样直接在车上应用么？还是只能在CARPLAY上使用 - sentiment: NEUTRAL - sentiment_score: {"Positive": 0.010348141193389893, "Negative": 0.09099732339382172, "Neutral": 0.8986498713493347, "Mixed": 4.764513505506329e-06}

你好，为什么在互联驾驶这个功能中定位不了我车了，显示说我在车上的GPS关闭了，但是我检查过了，也开着啊什么问题呢 - sentiment: NEGATIVE - sentiment_score: {"Positive": 0.0004973442992195487, "Negative": 0.9937689900398254, "Neutral": 0.005685589741915464, "Mixed": 4.8103760491358116e-05}

我喜欢使用互联驾驶这个功能中路线规划 - sentiment: POSITIVE - sentiment_score: {"Positive": 0.9950544834136963, "Negative": 5.20801440870855e-05, "Neutral": 0.004892551805824041, "Mixed": 8.196818157557573e-07}



2. 按照25条评论为一组，批量调用batch_detect_sentiment API，并把结果按照如下格式保存

```评论 | 情感分析结果 | 置信度```

In [74]:
# batch_detect_sentiment 
sentiment_result_file = 'sentiment_result.csv'

if os.path.isfile(sentiment_result_file):
    os.remove(sentiment_result_file)

with open(sentiment_result_file, 'a+') as sentiment_f:
    sentiment_f.write('comment' + '|' + 'sentiment' + '|' + 'sentiment_score' + '\n')

sentiment_chunks = [sentiment_input_with_def[x:x+25] for x in range(0, len(sentiment_input_with_def), 25)]
for chunk in sentiment_chunks:
    sentiment_response = comprehend_client.batch_detect_sentiment(TextList=chunk, LanguageCode='zh')
    #print(sentiment_response['ResultList'])
    with open(sentiment_result_file, 'a+') as sentiment_f:
        s_input_index = 0
        for s_input in chunk:
          sentiment_str = sentiment_response['ResultList'][s_input_index]['Sentiment']
          sentiment_score = json.dumps(sentiment_response['ResultList'][s_input_index]['SentimentScore'])
          sentiment_f.write(s_input + '|' + sentiment_str + '|' + sentiment_score + '\n')
          s_input_index +=1
sentiment_f.close

<function TextIOWrapper.close()>

3. 预览情感分析结果前5行，给出总体分析数

In [75]:
sentiment_data_df = pd.read_csv(sentiment_result_file, delimiter='|', index_col=None)
print(sentiment_data_df.head())

print('Analysis sentiment done for total item: ', len(sentiment_data_df))

    comment sentiment                                    sentiment_score
0     新的APP   NEUTRAL  {"Positive": 0.0007595693459734321, "Negative"...
1  good job  POSITIVE  {"Positive": 0.9861030578613281, "Negative": 0...
2    145666   NEUTRAL  {"Positive": 0.00029819051269441843, "Negative...
3        测试   NEUTRAL  {"Positive": 0.00022875443391967565, "Negative...
4   已经成功推荐了  POSITIVE  {"Positive": 0.9979491829872131, "Negative": 3...
Analysis sentiment done for total item:  7636


## 中文分词处理
1. 利用jieba分词进行处理
2. 准备常用的停用词表 - 用于过滤对于推理无关的代词，电话号码，邮箱，时间戳等，用于提高精度

In [77]:
import logging,os,jieba
#!wget https://cdc-code.s3.cn-north-1.amazonaws.com.cn/chineseStopWords.txt
def get_stopwords(StopWordFileName):
    logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',level=logging.INFO)  
      #加载停用词表 
    stopword_set = set()
    with open(StopWordFileName,'r',encoding="utf-8") as stopwords:
        for stopword in stopwords: 
            stopword_set.add(stopword.strip("\n"))  
    return stopword_set
    
def clear_timestamp(mystr):
    patterns = [r"\w{3} \w{3} \d{2} \d{1,2}:\d{1,2}:\d{1,2} \d{4}\s*",    #sun aug 19 13:02:10 2018
        r"\w{3}, \d{2} \w{3} \d{4} \d{1,2}:\d{1,2}:\d{1,2} \w{2}\s*",     #Sun, 19 Aug 2018 13:02:08 ET
        r"\d{4}-\d{1,2}-\d{1,2} \d{1,2}:\d{1,2}:\s*",                       #2018-11-01 09:35:
        r"\d{4}/\d{1,2}/\d{1,2}\s*",                                    #2018/9/1
        r"\d{1,2}/\d{1,2}/\d{4}\s*",                                    #9/1/2018
        r"\d{4}.\d{1,2}.\d{0,2}\s*",                                    #2018.9.1
        r"\d{1,2}.\d{1,2}.\d{4}\s*",                                    #9.1.2018
        r"\d{4}-\d{1,2}-\d{1,2} \d{1,2}:\d{1,2}:\d{4}/\d{1,2}/\d{1,2}\s*",      #2018-11-01 11:18:2018/10/31
        r"\d{1,2}:\d{1,2}:\d{4}/\d{1,2}/\d{1,2}\s*",      #2018-11-01 11:18:2018/10/31 21:09:08
        r"\d{1,2}:\d{1,2}:\d{1,2}\s*(AM|PM|am|pm)\s*",        #4:00:58 PM
        r"\d{1,2}:\d{1,2}:\d{1,2}\s*",                                     #21:09:08
        r"\d{1,2}:\d{1,2}\s*",                                     #21:09:08
        r"(\d{4})年(\d{1,2})月(\d{1,2})日\s*",                 #2018年10月5日
        r"(\d{2,4})年\s*",                 #2018年
        r"(\d{4})年(\d{1,2})月\s*",                 #2018年10月
        r":\s*([\da-zA_Z]+\/)+([a-zA-Z0-9\.]+)"                     #URL
        ]

    s = mystr

    for p in patterns:
        s = re.sub(p,'', s)

    s = s.strip()
    return s

def clear_email_phone_colon(mystr):
    patterns = [r"1[0-9]{10}", #mobile
        r"(\(0\d{2}\) \d{8})|(\(0\d{3}\) \d{7})|(\(0\d{3}\)-\d{8}$)|(\(0\d{2}\)\d{8})|(\(0\d{3}\)\d{7})|(\(0\d{3}\)\d{8}$)",                       #phone
        r"(0\d{2}-\d{8})|(0\d{3}-\d{7})|(0\d{3}-\d{8}$)|(\d{8})",                       #phone
        r"\w+([-+.]\w+)*@\w+([-.]\w+)*\.\w+([-.]\w+)*"    #email
        ]

    s = mystr

    for p in patterns:
        s = re.sub(p,'', s)

    s = s.strip()
    return s

# Debug code
file  = 'stop-words-test.txt'
with open(file) as f:
    lines = f.readlines()
    
labels = []
for line in lines:
    s = clear_email_phone_colon(line)
    print(s)

1005_!_RS_!_客户罗先生（车主黄先生的家人）致电，表示其i豪华型车辆，之前已经接到互联驾驶通知协议手机号更改成功的通知，但目前使用新协议手机号登陆云端互联APP后，首页不显示远程控制选项了（更改协议手机号之前可以正常使用），针对此问题，烦请互联驾驶部门跟进处理。 签订互联驾驶协议时登记的车主姓名：黄先生 协议手机号码：（原协议手机号） VIN：LBV5S3104HSN87123 登陆云端互联密码：990308ljq 手机型号&版本：IOS 最新 云端互联APP版本：最新 联系人：罗先生 联系电话：
1002_!_Carplay_!_ya_gao 2018-11-01 09:35: 客户杨先生通过在线客户平台反馈，表示在2018/9/1在淮安润宝行店内购买的 530Li 尊享型 豪华套装车辆，客户表示互联驾驶已经开通但是还是没有Apple Carplay的选项，已建议客户发送邮件，针对客户问题烦请互联驾驶人员跟进处理。 联系人/协议手机号码： 车主/联系人：杨先生 车架号：LBVKY5109JSP87501
1002_!_Carplay_!_ya_gao 2018-11-01 10:36:2018/11/1 7:53:14客户李先生（车主刘力先生的朋友）致电，表示其（2018.09.29）在（唐山中宝）购买了（525LI M），（10.09）登陆BMW云端互联APP，遇到（CARPLAY无显示、无法使用）问题，已建议客户发送截图至互联驾驶邮箱，针对此问题烦请跟进处理。未转接。协议手机号码：+86


### 使用分词

In [78]:
!pip install --upgrade pip
!pip install datetime

     |████████████████████████████████| 1.5 MB 616 kB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2.3
    Uninstalling pip-20.2.3:
      Successfully uninstalled pip-20.2.3


## 不同调用jieba的模式，用于迭代数据，找出最佳结果

jieba不同模式参考 https://github.com/fxsjy/jieba
    
当前最佳结果采用了第六种模式
    
1. with_stop_words_paddle_pseg - 使用停用词，并且启用paddle + pseg模式
2. without_stop_words_paddle_pseg - 不使用停用词，并且启用paddle + pseg模式
3. without_stop_words_cut - 不使用停用词，并且启用精确模式
4. with_stop_words_cut - 使用停用词，并且启用精确模式
5. without_stop_words_cut_paddle - 不使用停用词，并且启用paddle + 精确模式
6. with_stop_words_cut_paddle - 使用停用词，并且启用paddle + 精确模式

In [97]:
import jieba.posseg as pseg

def with_stop_words_paddle_pseg(mystr, word_counts):
    #get stopwords
    stopwords = get_stopwords('chinesestopwords_test.txt')
    #启用停用词过滤
    no_phone = clear_email_phone_colon(mystr)
    no_timestamp = clear_timestamp(no_phone)
    fenci = re.sub(r"[\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——一！，;:：。？、~@#￥%……&*（）]+", "", no_timestamp)
    words = pseg.cut(fenci,use_paddle=True) #paddle模式+posseg
    filled_words = set()
    for word, flag in words:
        #print('raw %s %s' % (word, flag))
        # 人名, 地名，机构，方位名词, 量词, 代词, 时间, 副词
        if (flag == 'nr' or flag == 'ns' or flag == 'PER' 
            or flag == 'LOC' or flag == 'ORG' or flag == 'f' or flag == 'r' or flag == 'q'
            or flag == 't' or flag == 'TIME' or flag == 'd'):
            #print('%s %s' % (word, flag))
            continue
        else:
            if word not in stopwords:          #不在停用词表中
                if len(word) == 1:
                    continue
                else:
                    word_counts[word] = word_counts.get(word, 0) + 1
                filled_words.add(word)
    return filled_words


def without_stop_words_paddle_pseg(mystr, word_counts):
    #get stopwords
    stopwords = get_stopwords('chinesestopwords_test.txt')
    #启用停用词过滤
    no_phone = clear_email_phone_colon(mystr)
    no_timestamp = clear_timestamp(no_phone)
    fenci = re.sub(r"[\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——一！，;:：。？、~@#￥%……&*（）]+", "", no_timestamp)
    words = pseg.cut(fenci,use_paddle=True) #paddle模式+posseg
    filled_words = set()
    for word, flag in words:
        #print('raw %s %s' % (word, flag))
        # 人名, 地名，机构，方位名词, 量词, 代词, 时间, 副词
        if (flag == 'nr' or flag == 'ns' or flag == 'PER' 
            or flag == 'LOC' or flag == 'ORG' or flag == 'f' or flag == 'r' or flag == 'q'
            or flag == 't' or flag == 'TIME' or flag == 'd'):
            #print('%s %s' % (word, flag))
            continue
        else:
            if len(word) == 1:
                continue
            else:
                word_counts[word] = word_counts.get(word, 0) + 1
        filled_words.add(word)
    return filled_words

def without_stop_words_cut(mystr, word_counts): 
    #get stopwords
    stopwords = get_stopwords('chinesestopwords_test.txt')
    #启用停用词过滤
    no_phone = clear_email_phone_colon(mystr)
    no_timestamp = clear_timestamp(no_phone)
    fenci = re.sub(r"[\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——一！，;:：。？、~@#￥%……&*（）]+", "", no_timestamp)
    words = jieba.cut(fenci,cut_all=False)
    filled_words = set()
    for word in words:
        #print('raw %s %s' % (word, flag))
        if len(word) == 1:
            continue
        else:
            word_counts[word] = word_counts.get(word, 0) + 1
        filled_words.add(word)
    return filled_words


def with_stop_words_cut(mystr, word_counts): 
    #get stopwords
    stopwords = get_stopwords('chinesestopwords_test.txt')
    #启用停用词过滤
    no_phone = clear_email_phone_colon(mystr)
    no_timestamp = clear_timestamp(no_phone)
    fenci = re.sub(r"[\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——一！，;:：。？、~@#￥%……&*（）]+", "", no_timestamp)
    words = jieba.cut(fenci,cut_all=False)
    filled_words = set()
    for word in words:
        #print('raw %s %s' % (word, flag))
        # 人名, 地名，机构，方位名词, 量词, 代词, 时间, 副词
        if word not in stopwords:          #不在停用词表中
            if len(word) == 1:
                continue
            else:
                word_counts[word] = word_counts.get(word, 0) + 1
            filled_words.add(word)
    return filled_words


def without_stop_words_cut_paddle(mystr, word_counts): 
    #get stopwords
    stopwords = get_stopwords('chinesestopwords_test.txt')
    #启用停用词过滤
    no_phone = clear_email_phone_colon(mystr)
    no_timestamp = clear_timestamp(no_phone)
    fenci = re.sub(r"[\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——一！，;:：。？、~@#￥%……&*（）]+", "", no_timestamp)
    words = jieba.cut(fenci,use_paddle=True) #paddle模式
    filled_words = set()
    for word in words:
        #print('raw %s %s' % (word, flag))
        if len(word) == 1:
            continue
        else:
            word_counts[word] = word_counts.get(word, 0) + 1
        filled_words.add(word)
    return filled_words


def with_stop_words_cut_paddle(mystr, word_counts):   
    #get stopwords
    stopwords = get_stopwords('chinesestopwords_test.txt')
    #启用停用词过滤
    no_phone = clear_email_phone_colon(mystr)
    no_timestamp = clear_timestamp(no_phone)
    fenci = re.sub(r"[\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——一！，;:：。？、~@#￥%……&*（）]+", "", no_timestamp)
    words = jieba.cut(fenci,cut_all=False,use_paddle=True) #paddle模式
    filled_words = set()
    for word in words:
        #print('raw %s %s' % (word, flag))
        # 人名, 地名，机构，方位名词, 量词, 代词, 时间, 副词
        if word not in stopwords:          #不在停用词表中
            if len(word) == 1:
                continue
            else:
                word_counts[word] = word_counts.get(word, 0) + 1
            filled_words.add(word)
    return filled_words

## 执行分词解析

In [99]:
import datetime

#单机并行分词
#jieba.enable_parallel(8)
#paddle模式，精确匹配需要关闭parallel
jieba.disable_parallel()
#启动paddle模式
jieba.enable_paddle()

begin = datetime.datetime.now()
    
labels = []
counts = {}                 #计数{word，frequency}
for line in input_with_category:
    label = []
    line = line.split('_!_')
    if line[0].strip('"').isdigit() and index_to_label[line[0].strip('"')]:
        label_code = index_to_label[line[0].strip('"')]
        label.append('__label__' + label_code)
        
        words_after_jieba = with_stop_words_cut_paddle(line[2], counts)
        
        label.extend(words_after_jieba)
        #print(label)
        labels.append(label)

shuffle(labels)

end = datetime.datetime.now()
print('data processing time %d' %(end - begin).seconds)
print(labels[0:5])

Paddle enabled successfully......
2020-11-05 11:19:10,677:DEBUG:Paddle enabled successfully......


data processing time 24
[['__label__意见建议-CONCERN&ADVICE__OTHER', '有待', '软件', '改进', '功能'], ['__label__其他/APP__OTHER'], ['__label__其他/APP__USABILITY/DIFFICULT TO USE', '实用'], ['__label__其他__OTHER', '还行'], ['__label__其他/APP__USABILITY/DIFFICULT TO USE', '很好']]


## 检查常见词词频

In [100]:
items = list(counts.items())
items.sort(key=lambda x: x[1], reverse=True)
for i in range(30):
    word, count = items[i]
    print("{:<10}{:>7}".format(word, count))

功能            618
车辆            599
宝马            431
互联            425
更新            363
远程            290
驾驶            287
信息            268
喜欢            240
很好            237
服务            234
显示            225
定位            176
不错            166
激活            163
云端            152
状态            147
界面            142
开通            134
推荐            131
体验            125
手机            122
实用            120
操控            116
软件            115
系统            112
会员            108
希望            104
朋友             99
品牌             97


## 切分训练集和验证集数据
1. 按照80：20比例进行切分

In [82]:
prefix = 'classification/blazingtext/bmwticket'
t_train_data = labels[0:int(len(labels)*0.8)]
t_validation_data = labels[int(len(labels)*0.8):]

In [83]:
#t_train_data[0:13]

2. 保存切分好的训练集和验证集数据

In [84]:
import csv
t_train_file = 'tt.train'
t_validation_file = 'tt.validation'

with open(t_train_file, 'w') as csvoutfile:
    csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
    csv_writer.writerows(t_train_data)
    
with open(t_validation_file, 'w') as csvoutfile:
    csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
    csv_writer.writerows(t_validation_data)

3. 上传到S3

In [85]:
%%time

t_train_channel = prefix + '/train'
t_validation_channel = prefix + '/validation'

sess.upload_data(path='tt.train', bucket=bucket, key_prefix=t_train_channel)
sess.upload_data(path='tt.validation', bucket=bucket, key_prefix=t_validation_channel)

s3_train_data = 's3://{}/{}'.format(bucket, t_train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, t_validation_channel)

s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

CPU times: user 63.1 ms, sys: 8.26 ms, total: 71.4 ms
Wall time: 401 ms


## 使用BlazingText算法进行评论分类

1. 加载SageMaker BlazingText container

In [87]:
region_name = boto3.Session().region_name
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

2020-11-05 09:03:17,293:WARNING:'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Using SageMaker BlazingText container: 390948362332.dkr.ecr.cn-north-1.amazonaws.com.cn/blazingtext:latest (cn-north-1)


2. 配置BlazingText预测模型和超参调优

In [88]:
t_bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.c4.4xlarge',
                                         train_volume_size = 120,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)
t_bt_model.set_hyperparameters(
    mode="supervised",
    epochs=20,
    min_count=2,
    learning_rate=0.1,
    vector_dim=10,
    early_stopping=True,
    patience=4,
    min_epochs=5,
    word_ngrams=1)

2020-11-05 09:29:20,306:WARNING:Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


3. 准备模型输入

In [89]:
t_train_data = sagemaker.inputs.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
t_validation_data = sagemaker.inputs.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')
t_data_channels = {'train': t_train_data, 'validation': t_validation_data}

2020-11-05 09:29:23,078:WARNING:'s3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
2020-11-05 09:29:23,079:WARNING:'s3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


4. 模型训练执行

In [91]:
t_bt_model.fit(inputs=t_data_channels, logs=True)

2020-11-05 09:46:32,206:INFO:Creating training-job with name: blazingtext-2020-11-05-09-46-32-206


2020-11-05 09:46:32 Starting - Starting the training job...
2020-11-05 09:46:34 Starting - Launching requested ML instances......
2020-11-05 09:48:03 Starting - Preparing the instances for training.........
2020-11-05 09:49:33 Downloading - Downloading input data
2020-11-05 09:49:33 Training - Downloading the training image...
2020-11-05 09:49:57 Uploading - Uploading generated training modelArguments: train
[11/05/2020 09:49:47 WARNING 140437587171136] Loggers have already been setup.
[11/05/2020 09:49:47 WARNING 140437587171136] Loggers have already been setup.
[11/05/2020 09:49:47 INFO 140437587171136] nvidia-smi took: 0.0252230167389 secs to identify 0 gpus
[11/05/2020 09:49:47 INFO 140437587171136] Running single machine CPU BlazingText training using supervised mode.
[11/05/2020 09:49:47 INFO 140437587171136] Processing /opt/ml/input/data/train/tt.train . File size: 0 MB
[11/05/2020 09:49:47 INFO 140437587171136] Processing /opt/ml/input/data/validation/tt.validation . File size:

## 执行推理

***注意在模型训练和调优达到需要的精度前，请勿执行模型推理的端点的部署***

1. 部署模型端点

In [94]:
t_text_classifier = t_bt_model.deploy(initial_instance_count = 1,instance_type = 'ml.c5.large')

2020-11-05 10:33:33,894:WARNING:Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
2020-11-05 10:33:33,897:INFO:Creating model with name: blazingtext-2020-11-05-09-46-32-206
2020-11-05 10:33:35,005:WARNING:Using already existing model: blazingtext-2020-11-05-09-46-32-206
2020-11-05 10:33:35,057:INFO:Creating endpoint with name blazingtext-2020-11-05-09-46-32-206


--------------!

2. 执行推理，利用输入数据中无分类的数据进行推理，精度要求85%以上

In [96]:
predict_category_result_file = 'predict_category_result.csv'
low_confidence = [] # confidence<confidence_thredhold
confidence_thredhold = 0.85

if os.path.isfile(predict_category_result_file):
    os.remove(predict_category_result_file)

with open(predict_category_result_file, 'a+') as predict_f:
    predict_f.write('category' + '|' + 'comment' + '|' + 'confidence' + '\n')

# #单机并行分词
# jieba.enable_parallel(8)
# #paddle模式，精确匹配需要关闭parallel
# #jieba.disable_parallel()
# #启动paddle模式
# jieba.enable_paddle()

for sentences in input_without_category:
    counts = {} 
    # using the same nltk tokenizer that we used during data preparation for training
    tokenized_sentences = [' '.join(with_stop_words_cut(sentences, counts))]
    #print('sentences: ', sentences, ' , jieba: ', tokenized_sentences)

    #payload = {"instances" : tokenized_sentences, "configuration": {"k": 2}}
    payload = {"instances" : tokenized_sentences}

    t_response = t_text_classifier.predict(json.dumps(payload))

    t_predictions = json.loads(t_response)
    #print(json.dumps(t_predictions, indent=2))
    predict_prob = t_predictions[0]['prob'][0]
    predict_category = t_predictions[0]['label'][0]
    if predict_prob < confidence_thredhold:
        low_confidence.append(sentences + '|' + predict_category + '|' + str(predict_prob) + '\n')
    
    with open(predict_category_result_file, 'a+') as predict_f:
        predict_f.write(sentences + '|' + predict_category + '|' + str(predict_prob) + '\n')

predict_f.close
print('total predict: ', len(input_without_category), ' , > ', confidence_thredhold, ' confidence: ', len(input_without_category) - len(low_confidence))

total predict:  637  , >  0.85  confidence:  637


3. 预测完毕，删除推理端点，节省费用

In [101]:
t_text_classifier.delete_endpoint()

2020-11-05 11:20:17,441:INFO:Deleting endpoint configuration with name: blazingtext-2020-11-05-09-46-32-206
2020-11-05 11:20:17,537:INFO:Deleting endpoint with name: blazingtext-2020-11-05-09-46-32-206
